# Binder-ready Category + Intent-based Q&A
User selects a Category (sheet) and provides a question. The notebook finds answers only in the selected category.

In [ ]:
# Install dependencies if needed (uncomment)
# !pip install pandas sentence-transformers scikit-learn ipywidgets openpyxl

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display

# -------------------------------
# Step 1: Load all sheets from Master Excel
# -------------------------------
file_path = 'master.xlsx'
sheet_names = ['Deposit', 'CRM', 'CKYC', 'Advance', 'Locker', 'SVS', 'Teller',
               'RTGS-NEFT-Bills-BG', 'TXN Related', 'Lien Related', 'Office AC',
               'Others', 'VNDRINVC_SSAM', 'TDS Related', 'CERSAI']

sheet_dfs = {}
xls = pd.ExcelFile(file_path)

for sheet in sheet_names:
    if sheet in xls.sheet_names:
        temp_df = pd.read_excel(file_path, sheet_name=sheet)
        temp_df.columns = [col.strip() for col in temp_df.columns]
        required_columns = ['Question/Issue', 'Solution']
        if all(col in temp_df.columns for col in required_columns):
            sheet_dfs[sheet] = temp_df
        else:
            print(f"Sheet '{sheet}' missing required columns, skipping.")
    else:
        print(f"Sheet '{sheet}' not found in file.")

print(f"Loaded sheets: {list(sheet_dfs.keys())}")

# -------------------------------
# Step 2: Load sentence-transformer model
# -------------------------------
model = SentenceTransformer('all-MiniLM-L6-v2')

# Precompute embeddings for each sheet
for sheet, df in sheet_dfs.items():
    df['embedding'] = df['Question/Issue'].astype(str).apply(lambda x: model.encode(x))

# -------------------------------
# Step 3: Define semantic search function per sheet
# -------------------------------
def get_solution_by_category(category, user_question, sheet_dfs, top_k=3):
    if category not in sheet_dfs:
        return f"Category '{category}' not found."
    df = sheet_dfs[category]
    if df.empty:
        return f"No data found in category '{category}'."
    user_emb = model.encode(user_question)
    embeddings = np.stack(df['embedding'].values)
    sims = cosine_similarity([user_emb], embeddings)[0]
    top_indices = sims.argsort()[-top_k:][::-1]
    solutions = []
    for idx in top_indices:
        solutions.append(f"{df.iloc[idx]['Solution']} (Category: {category})")
    return solutions

In [ ]:
# -------------------------------
# Step 4: Interactive widgets: Category + Question
# -------------------------------

category_dropdown = widgets.Dropdown(
    options=list(sheet_dfs.keys()),
    description='Category:'
)

question_box = widgets.Text(
    description='Question:',
    placeholder='Type your question here'
)

output_box = widgets.Output()
submit_btn = widgets.Button(description='Get Solution')

def on_submit(b):
    category = category_dropdown.value
    question = question_box.value
    response = get_solution_by_category(category, question, sheet_dfs)
    with output_box:
        output_box.clear_output()
        if isinstance(response, list):
            for sol in response:
                print('-', sol)
        else:
            print(response)

submit_btn.on_click(on_submit)
display(category_dropdown, question_box, submit_btn, output_box)